# 🎬 IMDb Recommender — Data Exploration & Validation

This notebook walks through the complete pipeline with **data quality validation** at every stage:

1. **Data Loading** — Load raw IMDb datasets
2. **Data Validation (Pandera)** — Schema validation on raw & merged data
3. **Exploration** — Understand distributions and quality
4. **Feature Engineering** — Build model-ready features
5. **Post-Feature Validation** — Validate the engineered dataset
6. **Model Training** — Fit and validate the recommender
7. **Recommendations** — Test with real queries (including tconst disambiguation)

> Run `make train` first to download the data, or execute the cells below to explore interactively.

In [ ]:
import sys
from pathlib import Path

# Add project root to path for imports
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandera as pa

plt.style.use("dark_background")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["font.size"] = 12

from src.ingestion.loader import load_and_merge_datasets
from src.features.engineering import run_feature_pipeline
from src.models.recommender import ContentRecommender
from src.validation.schemas import (
    TitleBasicsSchema,
    TitleRatingsSchema,
    MergedDatasetSchema,
    FilteredDatasetSchema,
    FeatureEngineeredSchema,
    validate_dataframe,
)

print(f"✅ Imports ready — Pandera v{pa.__version__}")


## 1. Load Raw Data

Load and merge `title.basics` and `title.ratings` from the raw data directory.

In [ ]:
raw_dir = project_root / "data" / "raw"
df = load_and_merge_datasets(raw_dir)
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
df.head(10)

## 2. Data Validation — Pandera Schema Check (Merged)

Validate the merged DataFrame against `MergedDatasetSchema` to catch data quality issues early:
- All required columns present with correct types
- `tconst` starts with `"tt"` and is unique
- `averageRating` ∈ [0, 10], `numVotes` > 0
- No unexpected nulls in critical columns

In [ ]:
# Validate merged dataset against Pandera schema
validate_dataframe(df, MergedDatasetSchema)

# Quick data quality summary
print(f"\n📋 Data Quality Summary:")
print(f"   Rows: {len(df):,}")
print(f"   Unique tconst: {df['tconst'].nunique():,}")
print(f"   Null genres: {df['genres'].isna().sum():,} ({df['genres'].isna().mean()*100:.1f}%)")
print(f"   Null runtime: {df['runtimeMinutes'].isna().sum():,} ({df['runtimeMinutes'].isna().mean()*100:.1f}%)")
print(f"   Rating range: [{df['averageRating'].min()}, {df['averageRating'].max()}]")
print(f"   Votes range: [{df['numVotes'].min():,}, {df['numVotes'].max():,}]")

## 3. Data Overview

Explore data types, distributions, and quality.

In [ ]:
# Title type distribution
print("📊 Title Type Distribution:")
print(df["titleType"].value_counts().to_string())
print(f"\n📈 Numerical Summary:")
df[["averageRating", "numVotes", "runtimeMinutes", "startYear"]].describe()

In [ ]:
# Genre frequency analysis
genres_exploded = (
    df["genres"]
    .dropna()
    .str.split(",")
    .explode()
    .str.strip()
    .value_counts()
    .head(20)
)

fig, ax = plt.subplots(figsize=(12, 8))
genres_exploded.sort_values().plot(kind="barh", ax=ax, color="#f5c518", edgecolor="none")
ax.set_title("Top 20 Genres in IMDb Dataset", fontsize=16, fontweight="bold", color="#f5c518")
ax.set_xlabel("Count", fontsize=12)
ax.set_ylabel("")
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()
plt.show()

# Rating distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].hist(df["averageRating"].dropna(), bins=50, color="#3498db", edgecolor="none", alpha=0.8)
axes[0].set_title("Rating Distribution", fontsize=14, fontweight="bold")
axes[0].set_xlabel("Average Rating")
axes[1].hist(np.log10(df["numVotes"].dropna() + 1), bins=50, color="#2ecc71", edgecolor="none", alpha=0.8)
axes[1].set_title("Vote Count Distribution (log10)", fontsize=14, fontweight="bold")
axes[1].set_xlabel("log10(numVotes)")
plt.tight_layout()
plt.show()

## 4. Feature Engineering

Run the full feature pipeline: filter → encode genres → scale numerics → build feature matrix.

In [ ]:
df_processed, feature_matrix, feature_names, artifacts = run_feature_pipeline(
    df=df,
    min_votes=500,
    min_year=1970,
)

print(f"\nProcessed DataFrame: {df_processed.shape}")
print(f"Feature matrix: {feature_matrix.shape}")
print(f"Features: {feature_names}")
df_processed[["tconst", "primaryTitle", "genres", "averageRating", "numVotes", "startYear"]].head(10)

## 5. Post-Feature Validation — Pandera Schema Check

Validate the engineered DataFrame against `FeatureEngineeredSchema`:
- Scaled features are in [0, 1]
- Genre one-hot columns exist
- No NaN in the feature matrix
- Dataset has enough rows for meaningful recommendations

In [ ]:
# Validate feature-engineered dataset
validate_dataframe(df_processed, FeatureEngineeredSchema)

# Verify feature matrix integrity
genre_cols = [c for c in df_processed.columns if c.startswith("genre_")]
scaled_cols = [c for c in df_processed.columns if c.endswith("_scaled")]

print(f"\n🔬 Feature Matrix Integrity:")
print(f"   Shape: {feature_matrix.shape}")
print(f"   dtype: {feature_matrix.dtype}")
print(f"   NaN count: {np.isnan(feature_matrix).sum()}")
print(f"   Scaled features ({len(scaled_cols)}): {scaled_cols}")
print(f"   Genre features ({len(genre_cols)}): {genre_cols[:5]}{'...' if len(genre_cols) > 5 else ''}")
print(f"   Value range: [{feature_matrix.min():.4f}, {feature_matrix.max():.4f}]")

## 6. Model Training & Validation

Fit the `ContentRecommender` and test with well-known titles.

In [ ]:
# Fit the recommender
model = ContentRecommender()
model.fit(
    feature_matrix=feature_matrix,
    tconst_ids=df_processed["tconst"].tolist(),
    titles=df_processed["primaryTitle"].tolist(),
)

# Test with Inception
print("🎬 Recommendations for 'Inception':\n")
recs = model.recommend("Inception", top_k=10)
pd.DataFrame(recs)

## 7. Explore More Recommendations

Test with various popular titles and demonstrate **tconst disambiguation** for duplicate title names.

In [ ]:
test_titles = ["The Dark Knight", "Breaking Bad", "Pulp Fiction", "The Matrix", "Stranger Things"]

for title in test_titles:
    try:
        recs = model.recommend(title, top_k=5)
        print(f"\n🎬 Top 5 for '{title}':")
        for r in recs:
            print(f"   #{r['rank']} {r['title']} (score: {r['similarity_score']:.4f})")
    except ValueError as e:
        print(f"\n⚠️  {title}: {e}")

# --- tconst disambiguation ---
# When multiple titles share the same name (e.g., "Parasite"),
# use the tconst ID to select the exact one you want.
print("\n" + "=" * 60)
print("🔍 tconst Disambiguation Demo:")
print("=" * 60)

# Find all titles named "Parasite"
parasite_matches = df_processed[df_processed["primaryTitle"] == "Parasite"]
if len(parasite_matches) > 0:
    print(f"\nFound {len(parasite_matches)} titles named 'Parasite':")
    for _, row in parasite_matches.iterrows():
        print(f"   {row['tconst']} — {row['primaryTitle']} ({int(row['startYear'])})")

    # Recommend using the specific 2019 Parasite via tconst
    target_tconst = parasite_matches.iloc[0]["tconst"]
    recs = model.recommend("Parasite", top_k=5, tconst=target_tconst)
    print(f"\n🎬 Recommendations for 'Parasite' (tconst={target_tconst}):")
    for r in recs:
        print(f"   #{r['rank']} {r['title']} (score: {r['similarity_score']:.4f})")
else:
    print("   'Parasite' not found in dataset")

## Summary

This notebook demonstrates the full IMDb Recommender pipeline with **data validation at every stage**:

| Stage | What | Validation |
|:-----:|------|-----------|
| 1 | **Data Loading** — Merged 2 IMDb datasets with proper null handling | — |
| 2 | **Pandera Validation** — Schema check on merged dataset | `MergedDatasetSchema` |
| 3 | **Exploration** — Analyzed genre, rating, and vote distributions | — |
| 4 | **Feature Engineering** — Genre encoding + numerical scaling → 30-D feature matrix | — |
| 5 | **Post-Feature Validation** — Schema check on engineered data | `FeatureEngineeredSchema` |
| 6 | **Model Training** — Content-based recommender with cosine similarity | Built-in `_validate_model()` |
| 7 | **Recommendations** — Tested with popular titles + tconst disambiguation | — |

**Available Pandera schemas** (in `src/validation/schemas.py`):
- `TitleBasicsSchema` — raw `title.basics.tsv`
- `TitleRatingsSchema` — raw `title.ratings.tsv`
- `MergedDatasetSchema` — merged basics + ratings
- `FilteredDatasetSchema` — after title type / votes / year filtering
- `FeatureEngineeredSchema` — final model-ready DataFrame

To serve this model via API: `make api`
To launch the UI: `make ui`
To view MLflow experiments: `make mlflow`